In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))
import base_experiment



ROOT_DIR, DATA_PATH, LOG_DIR, DEVICE = base_experiment.setup_experiment(
    seed=36, 
    data_path="dataset/pokec", 
    log_dir="experiments/fair_gnn/logs/pokec_z", 
    device=1
)

# import these after, as we need to set correct path in setup_experiment
import torch
import numpy as np
from models.fair import FairGNN, FairGNNTrainer
from dataset import PokecZ

Using directories:
root_dir: /home/fact21/fact_refactor
data_dir: /home/fact21/fact_refactor/dataset/pokec
log_dir: /home/fact21/fact_refactor/experiments/fair_gnn/logs/pokec_z
device: cuda:1


In [2]:
# Load in the dataset
dataset = PokecZ(
    nodes_path=DATA_PATH / "region_job.csv",
    edges_path=DATA_PATH / "region_job_relationship.txt",
    embedding_path=DATA_PATH / "pokec_z_embedding10.npy",
    feat_drop_rate=0.3,
    device=DEVICE
)

print(f"Loaded dataset with {dataset.graph.num_nodes()} nodes and {dataset.graph.num_edges()} edges")
print(f"Using feat_drop_rate: {dataset.feat_drop_rate}")

Loaded dataset with 67796 nodes and 1303712 edges
Using feat_drop_rate: 0.3


In [3]:
# Create FairGNN model
fair_gnn = FairGNN(
    num_features=dataset.features.shape[1],
    alpha=100, # for pokec_z, alpha is set to 100
).to(DEVICE)

# load pre-trained estimator
estimator_path = ROOT_DIR / "src/checkpoint/GCN_sens_region_job_ns_200"
fair_gnn.estimator.load_state_dict(torch.load(str(estimator_path.resolve())))

print(f"Created FairGNN model with {1} sensitive class")

Created FairGNN model with 1 sensitive class


In [4]:
# Create fair gnn trainer
trainer = FairGNNTrainer(
    dataset=dataset,
    fair_gnn=fair_gnn,
    device=DEVICE,
    log_dir=LOG_DIR,
    min_acc=0.65,
    min_roc=0.69,
)

print(f"Created trainer with {'GCN'} model, using LOG_DIR: {LOG_DIR}")

Created trainer with GCN model, using LOG_DIR: /home/fact21/fact_refactor/experiments/fair_gnn/logs/pokec_z


In [5]:
# Train the model
trainer.train(epochs=3000)

  0%|          | 0/3000 [00:00<?, ?it/s]

Finished training!

Best fair model:
	acc: 0.6571
	roc: 0.7180
	parity: 0.0241
	equality: 0.0417
